In [1]:
RUBY_CMD = 'ruby'
BUNDLE_CMD = 'bundle exec ruby'
META_CLI = File.absolute_path('/opt/OpenStudio-server/bin/openstudio_meta')
PROJECT = File.absolute_path('/opt/OpenStudio-server/server/spec/files')
HOST = '192.168.65.3'

"192.168.65.3"

In [2]:
@host = HOST
@project = PROJECT
@meta_cli = META_CLI
@ruby_cmd = RUBY_CMD
@bundle_cmd = BUNDLE_CMD

"bundle exec ruby"

In [3]:
command = "#{@meta_cli} run_analysis --debug --verbose '#{@project}/SEB_calibration_single_run_2013.json' 'http://#{@host}' -z 'SEB_calibration_NSGA_2013' -a single_run"

"/opt/OpenStudio-server/bin/openstudio_meta run_analysis --debug --verbose '/opt/OpenStudio-server/server/spec/files/SEB_calibration_single_run_2013.json' 'http://192.168.65.3' -z 'SEB_calibration_NSGA_2013' -a single_run"

In [4]:
run_analysis = system(command)

true